# Programmatically Specifying the Schema

In [16]:
#data import
import pandas as pd
data=pd.read_csv("data.csv")
data

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,0.009170,"['DJ Combo', 'Sander-7', 'Tony T']",0.792,147615,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,0,The One,0,2020-12-25,0.0356,125.972,0.1860,2020
174385,0.795000,['Alessia Cara'],0.429,144720,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.2280,2021
174386,0.806000,['Roger Fly'],0.671,218147,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,0,Together,0,2020-12-09,0.0282,108.058,0.7140,2020
174387,0.920000,['Taylor Swift'],0.462,244000,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,1,champagne problems,69,2021-01-07,0.0377,171.319,0.3200,2021


In [2]:
#data preprocessing
import os
import pyspark.sql.types as typ
import pyspark.sql.functions as F

from pyspark.sql import SparkSession
spark= SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
data_file = 'data.csv'

In [4]:
spotifyRDD = sc.textFile(data_file)

In [5]:
spotifyRDD.take(5)

['acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year',
 "0.991,['Mamie Smith'],0.598,168333,0.22399999999999998,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.379,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.634,1920",
 '0.643,"[""Screamin\' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.0264,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.95,1920',
 "0.993,['Mamie Smith'],0.647,163827,0.18600000000000005,0,11m7laMUgmOKqI3oYzuhne,1.76e-05,0,0.519,-12.097999999999999,1,Golfing Papa,4,1920,0.174,97.6,0.6890000000000001,1920",
 "0.000173,['Oscar Velazquez'],0.73,422087,0.7979999999999999,0,19Lc5SfJJ5O1oaxY0fpwfh,0.8009999999999999,2,0.128,-7.311,1,True House Music - Xavier Santos & Carlos Gomix Remix,17,1920-01-01,0.0425,127.99700000000001,0.0422,1920"]

In [6]:
type(spotifyRDD)

pyspark.rdd.RDD

In [7]:
header = spotifyRDD.first()

In [8]:
# store the comma-separated data in an RDD, skipping the header row. 
spotifyRDD = spotifyRDD \
        .filter(lambda row: row != header) \
        .map(lambda row: [elem for elem in row.split(',')]) 

In [9]:
type(spotifyRDD)

pyspark.rdd.PipelinedRDD

In [10]:
spotifyRDD.take(1)

[['0.991',
  "['Mamie Smith']",
  '0.598',
  '168333',
  '0.22399999999999998',
  '0',
  '0cS0A1fUEUd1EW3FcF8AEI',
  '0.000522',
  '5',
  '0.379',
  '-12.628',
  '0',
  'Keep A Song In Your Soul',
  '12',
  '1920',
  '0.0936',
  '149.976',
  '0.634',
  '1920']]

# 

In [11]:
header 

'acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year'

In [99]:
strings = ["artists", "danceability", "energy", "id", "name", "release_date"]

In [100]:
string_fields = [
    *[
        typ.StructField(h[0:], typ.StringType(), True)
        for h in header.split(',') if h in strings
    ]
]
#not sure if this is correct way to account for artist, id, name, release_date, year(?) 

In [101]:
string_fields

[StructField(acousticness,StringType,true),
 StructField(artists,StringType,true),
 StructField(danceability,StringType,true),
 StructField(energy,StringType,true),
 StructField(id,StringType,true),
 StructField(name,StringType,true),
 StructField(release_date,StringType,true)]

In [102]:
ints = ["duration_ms", "explicit", "key", "mode", "popularity", "year"]

In [103]:
int_fields = [
    *[
        typ.StructField(h[0:], typ.IntegerType(), True)
        for h in header.split(',') if h in ints
    ]
]

In [104]:
int_fields

[StructField(duration_ms,IntegerType,true),
 StructField(explicit,IntegerType,true),
 StructField(key,IntegerType,true),
 StructField(mode,IntegerType,true),
 StructField(popularity,IntegerType,true),
 StructField(year,IntegerType,true)]

In [105]:
flt_fields = [
    *[
        typ.StructField(h[0:], typ.FloatType(), True)
        for h in header.split(',') if h not in strings and h not in ints
    ]
]

In [106]:
flt_fields

[StructField(instrumentalness,FloatType,true),
 StructField(liveness,FloatType,true),
 StructField(loudness,FloatType,true),
 StructField(speechiness,FloatType,true),
 StructField(tempo,FloatType,true),
 StructField(valence,FloatType,true)]

In [107]:
#add into one list 
fields = string_fields + flt_fields + int_fields
fields

[StructField(acousticness,StringType,true),
 StructField(artists,StringType,true),
 StructField(danceability,StringType,true),
 StructField(energy,StringType,true),
 StructField(id,StringType,true),
 StructField(name,StringType,true),
 StructField(release_date,StringType,true),
 StructField(instrumentalness,FloatType,true),
 StructField(liveness,FloatType,true),
 StructField(loudness,FloatType,true),
 StructField(speechiness,FloatType,true),
 StructField(tempo,FloatType,true),
 StructField(valence,FloatType,true),
 StructField(duration_ms,IntegerType,true),
 StructField(explicit,IntegerType,true),
 StructField(key,IntegerType,true),
 StructField(mode,IntegerType,true),
 StructField(popularity,IntegerType,true),
 StructField(year,IntegerType,true)]

In [108]:
schema = typ.StructType(fields)

In [121]:
#create dataframe by passing schema
spotify_frame =spark.createDataFrame(spotifyRDD, schema)

In [110]:
spotify_frame.cache()

DataFrame[acousticness: string, artists: string, danceability: string, energy: string, id: string, name: string, release_date: string, instrumentalness: float, liveness: float, loudness: float, speechiness: float, tempo: float, valence: float, duration_ms: int, explicit: int, key: int, mode: int, popularity: int, year: int]

In [111]:
spotify_frame.printSchema()

root
 |-- acousticness: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- loudness: float (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- key: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- year: integer (nullable = true)



In [112]:
data.dtypes #actual data types of original csv

acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity            int64
release_date         object
speechiness         float64
tempo               float64
valence             float64
year                  int64
dtype: object

In [78]:
(data['acousticness'].map(type) == float).value_counts(dropna = False)

True    174389
Name: acousticness, dtype: int64

In [113]:
spotify_frame.show(5) 
#gives TypeError: 
#field acousticness: FloatType can not accept object '0.22399999999999998' in type <class 'str'>

#error means that a field in DataFrame’s schema is defined as a type that is different from the actual data type

Py4JJavaError: An error occurred while calling o306.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 12, 10.0.0.209, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Applications/miniconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/Applications/miniconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Applications/miniconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Applications/miniconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 107, in wrapper
    return f(*args,

# Directly creating spark dataframe

### Possible research question: Predict song popularity

In [3]:
import os
import pyspark.sql.types as typ
import pyspark.sql.functions as F

from pyspark.sql import SparkSession
spark= SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [4]:
data_file = 'data.csv'

In [5]:
spotify_df = spark.read.csv(data_file, header=True)

In [115]:
type(spotify_df)

pyspark.sql.dataframe.DataFrame

In [17]:
data.dtypes

acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity            int64
release_date         object
speechiness         float64
tempo               float64
valence             float64
year                  int64
dtype: object

In [55]:
(data['release_date'].map(type) == str).value_counts(dropna = False)

True    174389
Name: release_date, dtype: int64

In [39]:
#converting columns to appropriate types
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType

floats = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "loudness",
         "speechiness", "tempo", "valence"]
for i in floats:
        spotify_df = spotify_df.withColumn(i, spotify_df[i].cast(FloatType()))

ints = ["duration_ms", "explicit", "key", "mode", "popularity", "year"]
for i in ints:
    spotify_df = spotify_df.withColumn(i, spotify_df[i].cast(IntegerType()))

In [40]:
spotify_df

DataFrame[acousticness: float, artists: string, danceability: float, duration_ms: int, energy: float, explicit: int, id: string, instrumentalness: float, key: int, liveness: float, loudness: float, mode: int, name: string, popularity: int, release_date: string, speechiness: float, tempo: float, valence: float, year: int]

In [41]:
spotify_df.createOrReplaceTempView("spotify_df")

In [42]:
spark.sql("select * from spotify_df").collect()

[Row(acousticness=0.0, artists="['Mamie Smith']", danceability=0.5979999899864197, duration_ms=168333, energy=0.2240000069141388, explicit=0, id='0cS0A1fUEUd1EW3FcF8AEI', instrumentalness=0.000522000016644597, key=5, liveness=0.3790000081062317, loudness=-12.628000259399414, mode=0, name='Keep A Song In Your Soul', popularity=12, release_date='1920', speechiness=0.09359999746084213, tempo=149.9759979248047, valence=0.6340000033378601, year=1920),
 Row(acousticness=0.0, artists='"[""Screamin\' Jay Hawkins""]"', danceability=0.8519999980926514, duration_ms=150200, energy=0.5170000195503235, explicit=0, id='0hbkKFIJm7Z05H8Zl9w30f', instrumentalness=0.026399999856948853, key=5, liveness=0.08089999854564667, loudness=-7.261000156402588, mode=0, name='I Put A Spell On You', popularity=7, release_date='1920-01-05', speechiness=0.05339999869465828, tempo=86.88899993896484, valence=0.949999988079071, year=1920),
 Row(acousticness=0.0, artists="['Mamie Smith']", danceability=0.6470000147819519, 

In [43]:
spotify_df.printSchema()

root
 |-- acousticness: float (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: float (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- energy: float (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: float (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- release_date: string (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- year: integer (nullable = true)



In [44]:
spotify_df.show(5)

+------------+--------------------+------------+-----------+------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-------+-------+----+
|acousticness|             artists|danceability|duration_ms|energy|explicit|                  id|instrumentalness|key|liveness|loudness|mode|                name|popularity|release_date|speechiness|  tempo|valence|year|
+------------+--------------------+------------+-----------+------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-------+-------+----+
|         0.0|     ['Mamie Smith']|       0.598|     168333| 0.224|       0|0cS0A1fUEUd1EW3Fc...|         5.22E-4|  5|   0.379| -12.628|   0|Keep A Song In Yo...|        12|        1920|     0.0936|149.976|  0.634|1920|
|         0.0|"[""Screamin' Jay...|       0.852|     150200| 0.517|       0|0hbkKFIJm7Z05H8Zl...|          0.0264|  5|  

In [73]:
#id is not useful, name of the song is not useful, and release date can be replaced by year. drop all three
spotify_df = spotify_df.drop("id", "name", "release_date")

In [75]:
#artists is the only remaining categorical feature
spotify_df.show(5)

+------------+--------------------+------------+-----------+------+--------+----------------+---+--------+--------+----+----------+-----------+-------+-------+----+
|acousticness|             artists|danceability|duration_ms|energy|explicit|instrumentalness|key|liveness|loudness|mode|popularity|speechiness|  tempo|valence|year|
+------------+--------------------+------------+-----------+------+--------+----------------+---+--------+--------+----+----------+-----------+-------+-------+----+
|         0.0|          ['LolCat']|       0.792|     129287| 0.532|       0|             0.0|  2|    0.16| -15.461|   0|         1|      0.328|122.403|  0.545|1920|
|         0.0| ['Ignacio Corsini']|       0.648|     154240|0.0995|       0|           0.846| 11|   0.112| -22.429|   1|         0|      0.105| 71.978|  0.422|1921|
|         0.0| ['Mehmet Kemiksiz']|       0.685|     155063| 0.698|       0|         1.97E-6|  4|  0.0768|  -8.184|   0|         0|     0.0421|133.951|  0.723|1921|
|         

In [60]:
#count rows and unique rows
print('rows={}'.format(spotify_df.count()))
print('distinct rows={}'.format(spotify_df.distinct().count()))

rows=174389
distinct rows=172229


In [62]:
#drop all duplicates
spotify_df = spotify_df.dropDuplicates()


rows=172229


In [68]:
#check for nulls
from pyspark.sql.functions import isnan, when, count, col

spotify_df.select([count(when(isnan(c), c)).alias(c) for c in spotify_df.columns]).show()

+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+------------+-----------+-----+-------+----+
|acousticness|artists|danceability|duration_ms|energy|explicit| id|instrumentalness|key|liveness|loudness|mode|name|popularity|release_date|speechiness|tempo|valence|year|
+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+------------+-----------+-----+-------+----+
|           0|      0|           0|          0|     0|       0|  0|               0|  0|       0|       0|   0|   0|         0|           0|          0|    0|      0|   0|
+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+------------+-----------+-----+-------+----+



In [77]:
spotify_df.dropna().corr("energy", "instrumentalness")

-0.21249468595017823

In [46]:
spotify_df.dropna().corr("acousticness", "instrumentalness")

nan

In [36]:
data.iloc[:, 0:].corr()


,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
acousticness,1.000000,-0.263217,-0.089169,-0.750852,-0.208176,0.221956,-0.028028,-0.029654,-0.546639,0.064633,-0.396744,-0.022437,-0.223840,-0.166968,-0.607515
danceability,-0.263217,1.000000,-0.100757,0.204838,0.200842,-0.215589,0.026266,-0.110033,0.249541,-0.048358,0.123746,0.239962,0.005479,0.536713,0.159095
duration_ms,-0.089169,-0.100757,1.000000,0.060516,-0.033808,0.103621,0.002020,0.028942,0.019791,-0.046849,0.024717,-0.097838,-0.008182,-0.183199,0.105661
energy,-0.750852,0.204838,0.060516,1.000000,0.102561,-0.177750,0.035780,0.134815,0.779267,-0.056160,0.328939,-0.112616,0.266448,0.326418,0.540850
explicit,-0.208176,0.200842,-0.033808,0.102561,1.000000,-0.130609,0.005282,0.037288,0.106249,-0.062503,0.152545,0.353872,0.008075,-0.009275,0.151619
instrumentalness,0.221956,-0.215589,0.103621,-0.177750,-0.130609,1.000000,-0.004619,-0.047941,-0.317562,-0.056731,-0.300625,-0.133966,-0.068656,-0.219188,-0.114259
key,-0.028028,0.026266,0.002020,0.035780,0.005282,-0.004619,1.000000,-0.003368,0.025227,-0.127397,0.001951,0.009648,0.005009,0.025592,0.019194
liveness,-0.029654,-0.110033,0.028942,0.134815,0.037288,-0.047941,-0.003368,1.000000,0.062695,0.001677,-0.078959,0.122034,0.008586,-0.005781,-0.011852
loudness,-0.546639,0.249541,0.019791,0.779267,0.106249,-0.317562,0.025227,0.062695,1.000000,-0.019250,0.337194,-0.213504,0.217914,0.302520,0.465189
mode,0.064633,-0.048358,-0.046849,-0.056160,-0.062503,-0.056731,-0.127397,0.001677,-0.019250,1.000000,0.007652,-0.040711,0.002438,0.021592,-0.048922
